In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict

import os
import lxml

import xml.etree.ElementTree as ET

### Parsing XML directly (non-pandas)

In [9]:
directory = "AllPublicXML"
master_dict = defaultdict(list)

for folder in os.listdir(directory):
    folder_path = os.path.join(directory, folder)
    for index, file in enumerate(os.listdir(folder_path)):
        f = os.path.join(folder_path, file)
        print('\r'+'working on file '+f, end='', flush=True)
        
        tree = ET.parse(f)
        root = tree.getroot()
        
        #grabbing text by element 
        #really no way of knowing if the tree structure is consistent in all files
        
        #I guess use 'findall' if there's duplicate tags, 
        #and hopefully they are all contained in the same node
        #love like you've never been hurt
        
        
        #nct_id
        #directly from filepath, use for accurate count since all should be here
        master_dict['nct_id'].append(f[-15:-4])
        
        #all of these should only have one value
        #phase seems to be missing for some of these, can possibly be found in titles
        simple_grab = (['brief_title', 'official_title', 'source', 'overall_status', 'start_date', 
                        'completion_date', 'phase', 'study_type', 'enrollment'])
        
        #grab all "simple" elements
        for element in simple_grab:
            try:
                master_dict[element].append(root.find(element).text)
            except:
                master_dict[element].append('no data')
        
        
        #sponsors - have to findall for "agency" (additional level down) and collapse to string
        try:
            agency_list = [x.text for x in root.findall('./sponsors//agency')]
            master_dict['sponsors'].append('; '.join(agency_list))
        except:
            master_dict['sponsors'].append('no data')
            
        #multiple conditions so we want to get all of them
        try:
            agency_list = [x.text for x in root.findall('./condition')]
            master_dict['condition'].append('; '.join(agency_list))
        except:
            master_dict['condition'].append('no data')
            
        #intervention type - multiple, only want unique values
        try:
            intv_type = np.unique([x.text for x in root.findall('./intervention/intervention_type')])
            master_dict['intervention_type'].append('; '.join(intv_type))
        except:
            master_dict['intervention_type'].append('no data')
            
        #intervention name - multiple, only want unique values
        try:
            intv_name = np.unique([x.text for x in root.findall('./intervention/intervention_name')])
            master_dict['intervention_name'].append('; '.join(intv_name))
        except:
            master_dict['intervention_name'].append('no data')
            
        #multiple countries so we want to get all of them
        try:
            countries = [x.text for x in root.findall('./location_countries/country')]
            master_dict['countries'].append('; '.join(countries))
        except:
            master_dict['countries'].append('no data')

working on file AllPublicXML\NCT0000xxxx\NCT00003473.xml

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



working on file AllPublicXML\NCT0002xxxx\NCT00026416.xml

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



working on file AllPublicXML\NCT0563xxxx\NCT05630638.xml

In [10]:
for k in master_dict:
    print(k, len(master_dict[k]))

nct_id 434564
brief_title 434564
official_title 434564
source 434564
overall_status 434564
start_date 434564
completion_date 434564
phase 434564
study_type 434564
enrollment 434564
sponsors 434564
condition 434564
intervention_type 434564
intervention_name 434564
countries 434564


In [12]:
master_data = pd.DataFrame(master_dict)

In [13]:
master_data.to_csv('2022_12_07 Master Clintrials.csv', index=False)

### Creating "Drug Index", expand intervention_name and include nct_id

In [ ]:
intervention = []
intv_nct_id = []

for ind, trial_intv in enumerate(master_data['intervention_name']):
    intv_list = trial_intv.lower().split('; ')
    intv_id = [master_data['nct_id'][ind] for x in intv_list]
    intervention.append(intv_list)
    intv_nct_id.append(intv_id)
    
intervention = [item for sublist in intervention for item in sublist]
intv_nct_id = [item for sublist in intv_nct_id for item in sublist]

In [34]:
drug_index = pd.DataFrame()
drug_index['Intervention'] = intervention
drug_index['NCT ID'] = intv_nct_id

In [36]:
drug_index.to_csv('drug_index.csv', index=False)

### matching drugs to a min start date

#### what do you want here?

- intervention
- phase
- min start date
- second min start date (in case of error)
- nct id from min start date
- list of considered nct ids

#### What are you going to do about all of these permutations of drug names...

In [39]:
len(np.unique(intervention))

352207